In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import os


In [ ]:
data_path = os.path.join("..", "data")

In [ ]:
X = np.load(data_path + '/initial_data/function_7/initial_inputs.npy')
y = np.load(data_path + '/initial_data/function_7/initial_outputs.npy')
print(f"X is:{X}")
print(f"Y is :{y}")

In [ ]:
# add the week 1 input and  result to the array
X = np.append(X, [[0.151199, 0.033732, 0.730708, 0.344778, 0.023911, 0.533736]], axis=0)
y = np.append(y, [0.62260079], axis=0)

# add the week 2 input and result to the array
X = np.append(X, [[0.023952, 0.436365, 0.285535, 0.137788, 0.353289, 0.747301]], axis=0)
y = np.append(y, [1.515243229538322], axis=0)

# add the week 3 input and result to the array
X = np.append(X, [[0.025165, 0.343159, 0.324152, 0.113682, 0.321857, 0.924456]], axis=0)
y = np.append(y, [1.0711503490830157], axis=0)

# add the week 4 input and result to the array
X = np.append(X, [[0.003259, 0.392839, 0.309986, 0.034139, 0.323222, 0.843995]], axis=0)
y = np.append(y, [0.9336738018455746], axis=0)

# add the week 5 input and result to the array
X = np.append(X, [[0.005966, 0.408227, 0.235281, 0.257264, 0.364575, 0.611106]], axis=0)
y = np.append(y, [1.8597476940165774], axis=0)

# add the week 6 input and result to the array
X = np.append(X, [[0.012847, 0.337198, 0.189260, 0.238606, 0.359323, 0.557438]], axis=0)
y = np.append(y, [1.8838079567704533], axis=0)

# add the week 7 input and result to the array
X = np.append(X, [[0.005938, 0.357099, 0.254090, 0.226019, 0.373983, 0.567737]], axis=0)
y = np.append(y, [1.8684324311018121], axis=0)

# add the week 8 input and result to the array
X = np.append(X, [[0.005690, 0.361594, 0.232773, 0.260054, 0.435480, 0.530060]], axis=0)
y = np.append(y, [1.4390676532540159], axis=0)

# add the week 9 input and result to the array
X = np.append(X, [[0.006000, 0.370000, 0.240000, 0.245000, 0.390000, 0.560000]], axis=0)
y = np.append(y, [1.7512570814124735], axis=0)

# add the week 10 input and result to the array
X = np.append(X, [[0.013022, 0.372151, 0.206708, 0.284926, 0.256746, 0.578120]], axis=0)
y = np.append(y, [1.929694497214775], axis=0)

# add the week 11 input and result to the array
X = np.append(X, [[0.001165, 0.304194, 0.258543, 0.284629, 0.258669, 0.614834]], axis=0)
y = np.append(y, [2.249046395648729], axis=0)

# add the week 12 input and result to the array
X = np.append(X, [[0.015379, 0.223299, 0.253615, 0.317338, 0.159980, 0.596164]], axis=0)
y = np.append(y, [1.7333957368055208], axis=0)

print(f"X is:{X}")
print(f"Y is :{y}")

In [ ]:
print(f"max y is :{np.max(y)}")

In [ ]:
# Define GP model
kernel = Matern(nu=2.5, length_scale_bounds=(1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, normalize_y=True)
gp.fit(X, y)

In [ ]:
# UCB acquisition function
# week 6 changed kappa to 4.0 from 2.0
def ucb(x, gp, kappa=4.0):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return mu + kappa * sigma

In [ ]:
n_samples = 3000000  # You can adjust this based on your compute budget

# Generate random 6D points in [0, 1]^6
X_random = np.random.rand(n_samples, 6)

In [ ]:
ucb_values = np.array([ucb(x, gp) for x in X_random])


In [ ]:
best_idx = np.argmax(ucb_values)
# next_best_point = X_grid[best_idx]
next_best_point = X_random[best_idx]
print("Next best point to evaluate (UCB) rounded:", np.round(next_best_point,6))
print("Next best point to evaluate (UCB):", next_best_point)
pred_mean, pred_std = gp.predict(next_best_point.reshape(1, -1), return_std=True)
print("Predicted maximum value (mean):", pred_mean[0])

In [ ]:
# SVM model as surrogate
# Scale inputs
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train SVM surrogate
svm_surrogate = SVR(kernel='rbf', C=5, epsilon=0.1)
svm_surrogate.fit(X_scaled, y)

# Predict on new candidates
X_new = np.random.rand(10000000, 6)
X_new_scaled = scaler.transform(X_new)
y_pred = svm_surrogate.predict(X_new_scaled)

# Maximization: find best candidate
best_idx = np.argmax(y_pred)
best_candidate = X_new[best_idx]
best_predicted_value = y_pred[best_idx]

print("Best candidate input:", best_candidate)
print("Best candidate input rounded:", np.round(best_candidate, 6))
print("Predicted maximum value:", best_predicted_value)